In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from resnet_models import create_resnet152_model, create_resnet152v2_model
import wandb
import os
from wandb.keras import WandbCallback

2023-03-23 00:13:17.215590: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 00:13:20.469437: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-23 00:13:20.858770: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-23 00:13:20.858784: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

# Loading Data Set
Three sets are created: training, validation, and test. 
- Labels are generated based on the folder structure. Class name must correspond to the subfolder name.
- Loading in batches, of size 32, to reduce memory usage.
- Label mode is set to categorical, which means that the labels are encoded as a categorical vector.

Bilinear interploation is set to default. This specify the method used in the resizing procedure. By default aspect ratio is not perserved, i.e., the ratio between image width and height.

One hot encoding is utilized when label mode is set to categorical.


The image load documentation is available [here](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory) and an example is available [here](https://keras.io/api/data_loading/image/).

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory="dataset/train",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )
val_ds = keras.utils.image_dataset_from_directory(
    directory="dataset/val",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )
test_ds = keras.utils.image_dataset_from_directory(
    directory="dataset/test",
    labels='inferred',
    label_mode='categorical',
    shuffle=False,
    batch_size= 32,
    image_size=(224, 224)
    )

In [ ]:
# initialize wandb
wandb.init(project="ResNet152", config={"learning_rate": 0.001, "epochs": 30, "verbose": 1,"name": "ResNet152", "architecture": "ResNet152"})

# configs
cfg = wandb.config

# create the model
model = create_resnet152_model(len(train_ds.class_names))

In [ ]:
# list of callbacks
callbacks = [
            WandbCallback(mode="min", monitor="val_loss", save_graph=True),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=0.00001),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
          ]

In [ ]:
model.fit(train_ds, epochs=cfg.epochs, batch_size=32, verbose=cfg.verbose, validation_data=val_ds, callbacks=callbacks) # train the model

In [ ]:
wandb.finish()